<a href="https://colab.research.google.com/github/SuriyaPriya17/Intelligent_system/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community

In [3]:
!pip install langchain-experimental

In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.8 MB/s eta 0:00:00


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.1 MB/s eta 0:00:00


In [8]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from copy import deepcopy

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader("sample.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
semantic_chunks = text_splitter.split_documents(docs)

# manually apply overlap across those semantic chunks
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []

    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            # Get the last N characters of the previous chunk
            overlap = prev[-overlap_characters:]
            # Prepend to current chunk
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)

    return overlapped_chunks

# Apply overlap (adjust 200 to suit your context)
documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a ECC?")
print("Answer:", result["result"])


Pages loaded: 16


/tmp/ipython-input-3992661630.py:23: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/tmp/ipython-input-3992661630.py:46: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()
/tmp/ipython-input-3992661630.py:51: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpen



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: Context:
content: the transaction is from a registered vehicle, only public
ellipticcurvegroupG. keysgeneratedbyregisteredvehiclescanbestoredinthe
Authorized licensed use limited to: Mepco Schlenk Engineering College.Downloaded on November 08,2024 at 03:54:58 UTC from IEEE Xplore. Restrictions apply.
source:sample.pdf

Context:
content:Domingos, L.Villas, and A. Boukerche, “Data communication in [22] Z.Xu,W.Liang,K.-C.Li,J.Xu,andH.Jin,“Ablockchain-basedroadside
VANETs:Survey,applicationsandchallenges,”AdHocNetw.,vol.44,
unit-assisted authentication and key agreement protocol for Internet of
pp.90–103,2016. Vehicles,”J.ParallelDistrib.Comput.,vol.149,pp.29–39,2021. [2] J.B.Kenney,“Dedicatedshort-rangecommunications(DSRC)